In [8]:
import numpy as np
import scipy.misc
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline

In [148]:
import generate_state_matrices
def states(*args, **kwargs):
    S, T, R = generate_state_matrices.states(*args, **kwargs)
    T = np.rollaxis(T, 2, 0)
    return S, T, R

import mdptoolbox, mdptoolbox.example
def backward_induction(T, R, N):
    fh = mdptoolbox.mdp.FiniteHorizon(T, R, 1, N=N)
    fh.run()
    return fh.V[:, 0]

from joblib import Memory
cache = Memory(cachedir='.joblib_cache', verbose=0).cache

In [161]:
S,T,R = states(2, 2, cost=0.01, constantArm=0.5)
backward_induction(T, R, 4)

array([ 0.61      ,  0.66666667,  0.57333333,  0.66666667,  0.57333333,
        0.75      ,  0.5       ,  0.66666667,  0.66666667,  0.5       ,
        0.66666667,  0.5       ,  0.75      ,  0.5       ,  0.5       ,  0.        ])

In [17]:
def expected_values(s):
    hits = s[0:None:2]
    misses = s[1:None:2]
    return hits/(misses+hits)

array([[-0.05      ,  0.5       ,  0.        ],
       [-0.05      ,  0.66666667,  0.        ],
       [-0.05      ,  0.33333333,  0.        ],
       [-0.05      ,  0.75      ,  0.        ],
       [-0.05      ,  0.5       ,  0.        ],
       [-0.05      ,  0.25      ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [200]:
R1[:, -1]

array([ 0.5       ,  0.66666667,  0.33333333,  0.75      ,  0.5       ,
        0.25      ,  0.8       ,  0.6       ,  0.4       ,  0.2       ,
        0.83333333,  0.66666667,  0.5       ,  0.33333333,  0.16666667,
        0.85714286,  0.71428571,  0.57142857,  0.42857143,  0.28571429,
        0.14285714,  0.875     ,  0.75      ,  0.625     ,  0.5       ,
        0.375     ,  0.25      ,  0.125     ,  0.88888889,  0.77777778,
        0.66666667,  0.55555556,  0.44444444,  0.33333333,  0.22222222,
        0.11111111,  0.9       ,  0.8       ,  0.7       ,  0.6       ,
        0.5       ,  0.4       ,  0.3       ,  0.2       ,  0.1       ,  0.        ])

0.5

In [204]:

backward_induction(T1, R1, 100)

array([ 0.5       ,  0.66666667,  0.33333333,  0.75      ,  0.5       ,
        0.25      ,  0.8       ,  0.6       ,  0.4       ,  0.2       ,
        0.83333333,  0.66666667,  0.5       ,  0.33333333,  0.16666667,
        0.85714286,  0.71428571,  0.57142857,  0.42857143,  0.28571429,
        0.14285714,  0.875     ,  0.75      ,  0.625     ,  0.5       ,
        0.375     ,  0.25      ,  0.125     ,  0.88888889,  0.77777778,
        0.66666667,  0.55555556,  0.44444444,  0.33333333,  0.22222222,
        0.11111111,  0.9       ,  0.8       ,  0.7       ,  0.6       ,
        0.5       ,  0.4       ,  0.3       ,  0.2       ,  0.1       ,  0.        ])

In [226]:
COST = 0.05
MAX_STEPS = int(1/(4*COST)) + 3
S1, T1, R1 = states(MAX_STEPS, 1, cost=COST, constantArm=-1)
R1_alt = R1.copy()

def s_idx(s):
    return int(sum(range(int(sum(s)-1))) + s[1] - 1)

# @cache
def V1(constant):
    R1_alt[:, -1] = np.maximum(constant, R1[:, -1])
    R1_alt[-1, -1] = 0
    return backward_induction(T1, R1_alt, N=MAX_STEPS)

def Q1(s, a, constant):
    if a == 1:  # bet
        return max(s[0] / sum(s), constant)
    else:  # observe
        V = V1(constant)
        return T1[a, s_idx(s)] @ V - COST

N_ARM = 2
S, T, R = states(MAX_STEPS, N_ARM, cost=COST)

def Q_blinker(s, a):
    s = np.array(s)
    if a == N_ARM:
        return max(expected_values(s))

    # alternative is selecting current best
    mu = expected_values(s)
    mu[a] = -np.inf
    alternative = mu.max()
    
    idx = a * 2
    s_arm = s[[idx, idx+1]]
    return Q1(s_arm, 0, alternative)


Q_blinker([1,1,1,1], 1)

0.75

In [28]:
s, a = 0, 0
idx = a * 2
s_arm = to_int_s(S[s][[idx, idx+1]])

S[s][[idx, idx+1]]
# mu = expected_values(S[s])
# mu[s] = -np.inf
# alternative = mu.max()


array([ 1.,  1.])

In [ ]:
def Q1(s, a, constant=0):
    if a == 1:  # bet on the one arm
        return max(s[0] / s.sum(), constant)
    else:  # observe
        alternative = s[1] / s.sum()  
        V = V1(alternative)
        return T[a, s] @ V - COST


In [287]:
def Q(s, a):
    return R[s, a] + T[a, s] @ V

In [ ]:
def Q_one()

In [205]:


def Q(s, a):
    
    
    
# print('recursion', V(2))
print('induction', backward_induction(T, R))

induction 0.588333333333


In [ ]:
from functools import lru_cache

n_actions = len(R[1])
actions = range(n_actions)
final_state = len(T) - 1

@lru_cache(None)
def V(s):
    if s == final_state:
        return 0
    return max(Q(s, a) for a in actions)

@lru_cache(None)
def Q(s, a):
    s1_options = T[s, :, a].nonzero()[0]
    s1_p = T[s, s1_options, a]
    return R[s, a] + sum(p * V(s1) for s1, p in zip(s1_options, s1_p))

In [193]:
s = states(4,2,cost=0.01)
a = {'states':s[0],'transition':s[1],'rewards':s[2]}
scipy.io.savemat('./file',a)